# Imports

In [ ]:
%pip install contractions
%pip install textcleaner
%pip install symspellpy
%pip install simpletransformers

In [69]:
import pandas as pd
from pandas import Timestamp
import numpy as np
from operator import itemgetter
from datetime import datetime
import re
from scipy import sparse

import matplotlib.pyplot as plt


import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

import simpletransformers
from simpletransformers.classification import ClassificationModel
import logging


import contractions
import textcleaner as tc

from sklearn.feature_extraction.text import CountVectorizer

from nltk.corpus import words

import pkg_resources
from symspellpy import SymSpell, Verbosity

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


# Data Look

In [2]:
# df_train= pd.read_csv('train.csv')
# df_test= pd.read_csv('test.csv')

df_train= pd.read_csv('/Users/francesco/Kaggle/train.csv')
df_test= pd.read_csv('/Users/francesco/Kaggle/test.csv')

In [3]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
df_train.shape

(7613, 5)

In [5]:
df_test.shape

(3263, 4)

In [6]:
df_train.location.value_counts()

USA                          104
New York                      71
United States                 50
London                        45
Canada                        29
                            ... 
Hamilton, Ontario Canada       1
aggressive cannoli eater       1
YA MOTHA BED                   1
Atlanta g.a.                   1
Eddyville, Oregon 97343        1
Name: location, Length: 3341, dtype: int64

In [7]:
df_train.keyword.value_counts()

fatalities               45
armageddon               42
deluge                   42
body%20bags              41
damage                   41
                         ..
forest%20fire            19
epicentre                12
threat                   11
inundation               10
radiation%20emergency     9
Name: keyword, Length: 221, dtype: int64

In [8]:
df_train.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [10]:
df_train.fillna('', inplace=True)
df_test.fillna('', inplace=True)

In [11]:
df_train.head()

,id,keyword,location,text,target
0,1,,,Our Deeds are the Reason of this #earthquake M...,1
1,4,,,Forest fire near La Ronge Sask. Canada,1
2,5,,,All residents asked to 'shelter in place' are ...,1
3,6,,,"13,000 people receive #wildfires evacuation or...",1
4,7,,,Just got sent this photo from Ruby #Alaska as ...,1


In [12]:
df_test.head()

,id,keyword,location,text
0,0,,,Just happened a terrible car crash
1,2,,,"Heard about #earthquake is different cities, s..."
2,3,,,"there is a forest fire at spot pond, geese are..."
3,9,,,Apocalypse lighting. #Spokane #wildfires
4,11,,,Typhoon Soudelor kills 28 in China and Taiwan


In [13]:
df_train['full_text'] = df_train['keyword'] + ' ' + df_train['location'] +  ' ' + df_train['text']
df_test['full_text'] = df_test['keyword'] + ' ' + df_test['location'] +  ' ' + df_test['text']

# Pre-Proccessing

In [17]:
'''Text Cleaning and Correcting'''

def replace_contractions(text):
    '''Replace words such as 'they're to they are' '''
    return contractions.fix(text)

def clean(text):
    '''Clean the text.'''
    try:
        cleaned_text = replace_contractions(text)
    except:
        print(text) 
        cleaned_text = None 
        pass
    
    cleaned_text = tc.document(cleaned_text).clear_blank_lines().strip_all().lower_all().remove_numbers()
    return cleaned_text.data

df_train.full_text = df_train.full_text.apply(lambda x: clean(x)) # Clean the text.
df_test.full_text = df_test.full_text.apply(lambda x: clean(x)) # Clean the text.


In [18]:
df_train.head()

,id,keyword,location,text,target,full_text
0,1,,,Our Deeds are the Reason of this #earthquake M...,1,[ our deeds are the reason of this #earthquake...
1,4,,,Forest fire near La Ronge Sask. Canada,1,[ forest fire near la ronge sask. canada]
2,5,,,All residents asked to 'shelter in place' are ...,1,[ all residents asked to 'shelter in place' ar...
3,6,,,"13,000 people receive #wildfires evacuation or...",1,"[ , people receive #wildfires evacuation order..."
4,7,,,Just got sent this photo from Ruby #Alaska as ...,1,[ just got sent this photo from ruby #alaska a...


In [19]:
df_test.head()

,id,keyword,location,text,full_text
0,0,,,Just happened a terrible car crash,[ just happened a terrible car crash]
1,2,,,"Heard about #earthquake is different cities, s...","[ heard about #earthquake is different cities,..."
2,3,,,"there is a forest fire at spot pond, geese are...","[ there is a forest fire at spot pond, geese a..."
3,9,,,Apocalypse lighting. #Spokane #wildfires,[ apocalypse lighting. #spokane #wildfires]
4,11,,,Typhoon Soudelor kills 28 in China and Taiwan,[ typhoon soudelor kills in china and taiwan]


In [20]:
'''Contractions turns the Note column into a list, lets turn that back into strings'''

df_train.full_text = df_train.full_text.astype(str).str.replace('[_/.]', ' ') # Turn list into strings.
df_train.full_text = df_train.full_text.astype(str).str.replace('["\[\]?!:,\'\(\)\{\}#@]', '') # Remove symbols.

df_test.full_text = df_test.full_text.astype(str).str.replace('[_/.]', ' ') # Turn list into strings.
df_test.full_text = df_test.full_text.astype(str).str.replace('["\[\]?!:,\'\(\)\{\}#@]', '') # Remove symbols.

df_train.head()
df_test.head()

,id,keyword,location,text,full_text
0,0,,,Just happened a terrible car crash,just happened a terrible car crash
1,2,,,"Heard about #earthquake is different cities, s...",heard about earthquake is different cities st...
2,3,,,"there is a forest fire at spot pond, geese are...",there is a forest fire at spot pond geese are...
3,9,,,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting spokane wildfires
4,11,,,Typhoon Soudelor kills 28 in China and Taiwan,typhoon soudelor kills in china and taiwan


In [21]:
'''Lets try to spell correct as many words as we can'''

words = set(nltk.corpus.words.words()) #Set of words in the english dict (missing some)

In [42]:
'''Number of individual words'''
cvec_train = CountVectorizer() # Count vectorizer.
cvec_train.fit(df_train.full_text)
cvec_mat_all_train = cvec_train.transform(df_train.full_text)
words_cvec_train = cvec_train.get_feature_names()

print(len(words_cvec_train)) #Number of individual words
      
      
'''Place the non-english words into a list so we can spell correct them'''
words_not_en_train = []
for word in words_cvec_train:
    '''Find words not in d'''
    if word not in words:        
        words_not_en_train.append(word)
      
print(len(words_not_en_train)) #This many words to spell correct

14629
6206


In [43]:
'''Number of individual words'''
cvec_test = CountVectorizer() # Count vectorizer.
cvec_test.fit(df_test.full_text)
cvec_mat_all_test = cvec_test.transform(df_test.full_text)
words_cvec_test = cvec_test.get_feature_names()

print(len(words_cvec_test)) #Number of individual words
      
      
'''Place the non-english words into a list so we can spell correct them'''
words_not_en_test = []
for word in words_cvec_test:
    '''Find words not in d'''
    if word not in words:        
        words_not_en_test.append(word)
      
print(len(words_not_en_test)) #This many words to spell correct

12936
8325


In [44]:
'''SymSpell checks for words which are either misspelled, or two words without a space between them.'''

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", 
                                                  "frequency_dictionary_en_82_765.txt"
                                                 )

bigram_path = pkg_resources.resource_filename("symspellpy", 
                                              "frequency_bigramdictionary_en_243_342.txt"
                                             )

sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)
sym_spell.load_bigram_dictionary(bigram_path, term_index=0, count_index=2)

True

In [45]:
def spell_check(text):  
    '''Spell checking using SymSpell.'''
    suggestion = sym_spell.lookup_compound(text, max_edit_distance = 2)[0]
    return suggestion.term, suggestion.count

In [46]:
'''Check our words_not_en'''
corrected_words_train = []

for word in words_not_en_train:
    '''Correct the words which can be corrected.'''
    corrected_words_train.append(spell_check(word))
    
    
corrected_words_test = []

for word in words_not_en_test:
    '''Correct the words which can be corrected.'''
    corrected_words_test.append(spell_check(word))

In [47]:
corrected_words_train[0:10]

[('aarhus', 596188),
 ('aaron', 11698784),
 ('abadan', 38580),
 ('abandoning', 778900),
 ('abba', 1670093),
 ('abbott', 3143893),
 ('abcs', 474819),
 ('abe', 2420884),
 ('aberdeen', 5660658),
 ('alerts with', 70564)]

In [48]:
corrected_words_test[0:10]

[('a axis', 109867),
 ('aabjgwfdv', 0),
 ('a dark', 142050560),
 ('jaffna can', 255),
 ('law toys', 18706),
 ('yahoo vital', 2152),
 ('papa work', 1706),
 ('aaron', 11698784),
 ('aaron goodwin', 19),
 ('aaronkearneyaus', 0)]

In [49]:
'''Place the old and new words into a dictionary.'''

word_change_dict_train = dict(zip(words_not_en_train, [word[0] if word[1] != 0 else '' for word in corrected_words_train])) 

word_change_dict_test = dict(zip(words_not_en_test, [word[0] if word[1] != 0 else '' for word in corrected_words_test])) 


In [178]:
'''We now have a dictionary of the non english words and their spelling corrections'''
# word_change_dict_train

'We now have a dictionary of the non english words and their spelling corrections'

In [179]:
'''We now have a dictionary of the non english words and their spelling corrections'''
# word_change_dict_test

'We now have a dictionary of the non english words and their spelling corrections'

In [56]:
'''Now we run through the Note column and replace the misspelled words'''

def spell_correct(text, word_dict):
    '''Search the text column strings and replace words which are mispelled.'''
    string = ''
    for word in text.split():
        try:
            new_word =  word_dict[word]
            string = string + new_word + ' '

        except:
            new_word = word
            string = string + new_word + ' '
            
    return string

df_train.full_text = df_train.full_text.apply(lambda x: spell_correct(x, word_change_dict_train))
df_test.full_text = df_test.full_text.apply(lambda x: spell_correct(x, word_change_dict_test))

In [57]:
df_train.head()

,id,keyword,location,text,target,full_text
0,1,,,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...
1,4,,,Forest fire near La Ronge Sask. Canada,1,forest fire near la range sask canada
2,5,,,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...
3,6,,,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in ...
4,7,,,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...


In [58]:
df_test.head()

,id,keyword,location,text,full_text
0,0,,,Just happened a terrible car crash,just happened a terrible car crash
1,2,,,"Heard about #earthquake is different cities, s...",heard about earthquake is different cities sta...
2,3,,,"there is a forest fire at spot pond, geese are...",there is a forest fire at spot pond geese are ...
3,9,,,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting spokane wildfires
4,11,,,Typhoon Soudelor kills 28 in China and Taiwan,typhoon so decor kills in china and taiwan


In [59]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/francesco/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [60]:
'''Lemment the words'''

lemmatizer = WordNetLemmatizer() 

def lemm(text):
    '''Lemment words in text column.'''
    return lemmatizer.lemmatize(text)

df_train.full_text = df_train.full_text.apply(lambda x: lemm(x))
df_test.full_text = df_test.full_text.apply(lambda x: lemm(x))

In [61]:
df_train.head()

,id,keyword,location,text,target,full_text
0,1,,,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...
1,4,,,Forest fire near La Ronge Sask. Canada,1,forest fire near la range sask canada
2,5,,,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...
3,6,,,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in ...
4,7,,,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...


In [62]:
df_test.head()

,id,keyword,location,text,full_text
0,0,,,Just happened a terrible car crash,just happened a terrible car crash
1,2,,,"Heard about #earthquake is different cities, s...",heard about earthquake is different cities sta...
2,3,,,"there is a forest fire at spot pond, geese are...",there is a forest fire at spot pond geese are ...
3,9,,,Apocalypse lighting. #Spokane #wildfires,apocalypse lighting spokane wildfires
4,11,,,Typhoon Soudelor kills 28 in China and Taiwan,typhoon so decor kills in china and taiwan


# Modelling

In [ ]:
'''Modelling'''

In [105]:
df_train.full_text.isnull().sum()

0

In [106]:
df_train.target.value_counts(normalize=True)

0    0.57034
1    0.42966
Name: target, dtype: float64

In [107]:
df_train.head()

,id,keyword,location,text,target,full_text
0,1,,,Our Deeds are the Reason of this #earthquake M...,1,our deeds are the reason of this earthquake ma...
1,4,,,Forest fire near La Ronge Sask. Canada,1,forest fire near la range sask canada
2,5,,,All residents asked to 'shelter in place' are ...,1,all residents asked to shelter in place are be...
3,6,,,"13,000 people receive #wildfires evacuation or...",1,people receive wildfires evacuation orders in ...
4,7,,,Just got sent this photo from Ruby #Alaska as ...,1,just got sent this photo from ruby alaska as s...


In [108]:
df_train_model = df_train[['full_text', 'target']]

In [109]:
df_train_model.head()

,full_text,target
0,our deeds are the reason of this earthquake ma...,1
1,forest fire near la range sask canada,1
2,all residents asked to shelter in place are be...,1
3,people receive wildfires evacuation orders in ...,1
4,just got sent this photo from ruby alaska as s...,1


In [110]:
# self.args = {
#    'model_type':  'roberta',
#    'model_name': 'roberta-base',
#    'output_dir': 'outputs/',
#    'cache_dir': 'cache/',
#    'fp16': True,
#    'fp16_opt_level': 'O1',
#    'max_seq_length': 128,
#    'train_batch_size': 8,
#    'eval_batch_size': 8,
#    'gradient_accumulation_steps': 1,
#    'num_train_epochs': 1,
#    'weight_decay': 0,
#    'learning_rate': 4e-5,
#    'adam_epsilon': 1e-8,
#    'warmup_ratio': 0.06,
#    'warmup_steps': 0,
#    'max_grad_norm': 1.0,
#    'logging_steps': 50,
#    'evaluate_during_training': False,
#    'save_steps': 2000,
#    'eval_all_checkpoints': True,
#    'use_tensorboard': True,
#    'overwrite_output_dir': False,
#    'reprocess_input_data': False,
# }

In [111]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [112]:
df_train_model.shape

(7613, 2)

In [115]:
model = ClassificationModel('roberta', 'roberta-base', use_cuda=True,
                          args={'num_train_epochs': 5, 'overwrite_output_dir': True})

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [ ]:
# Train the model 
model.train_model(df_train_model, args={'fp16': False})

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/simpletransformers/classification/classification_model.py:304: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


/anaconda/envs/azureml_py36/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


In [ ]:
# import sklearn

# result, model_outputs, wrong_predictions = model.eval_model(test, acc=sklearn.metrics.accuracy_score)

# Predicting

In [70]:
'''Import the BERT model'''

model = ClassificationModel('roberta', 'outputs_1', use_cuda=False,
                          args={'num_train_epochs': 5, 'overwrite_output_dir': False
                               })

Some weights of the model checkpoint at outputs_1 were not used when initializing RobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [71]:
model

In [134]:
df_test.full_text.isnull().sum()

0

In [135]:
df_test_model = df_test[['full_text']]

In [136]:
df_test_model.head()

,full_text
0,just happened a terrible car crash
1,heard about earthquake is different cities sta...
2,there is a forest fire at spot pond geese are ...
3,apocalypse lighting spokane wildfires
4,typhoon so decor kills in china and taiwan


In [137]:
df_test_model

,full_text
0,just happened a terrible car crash
1,heard about earthquake is different cities sta...
2,there is a forest fire at spot pond geese are ...
3,apocalypse lighting spokane wildfires
4,typhoon so decor kills in china and taiwan
...,...
3258,earthquake safety los angeles \x89ûò safety fa...
3259,storm in i worse than last hurricane my city&a...
3260,green line derailment in chicago help t co tub...
3261,meg issues hazardous weather outlook who help ...


In [138]:
df_test_model_list = df_test_model.full_text.values.tolist()

In [139]:
len(df_test_model_list)

3263

In [140]:
predictions = model.predict(df_test_model_list)

In [180]:
# predictions

In [142]:
len(predictions[0])

3263

In [181]:
# predictions[0]

In [182]:
# df_test.id

In [144]:
df_predictions_target = pd.DataFrame(predictions[0], columns = ['target'])

In [183]:
# df_predictions_target

In [154]:
df_predictions_id = pd.DataFrame(df_test.id, columns = ['id'])

In [155]:
df_predictions_id.head()

,id
0,0
1,2
2,3
3,9
4,11


In [163]:
df_predictions = pd.concat([df_predictions_id, df_predictions_target], axis=1)

In [184]:
# df_predictions

In [167]:
df_predictions.reset_index(inplace=True, drop=True)

In [168]:
df_predictions.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1


In [169]:
len(df_predictions)

3263

In [170]:
df_predictions.columns

Index(['id', 'target'], dtype='object')

In [177]:
df_predictions.to_csv('submission.csv', index=False)